In [ ]:
# %pip install --force-reinstall ../optpricerpy/dist/optpricerpy-0.1.0-cp39-cp39-manylinux_2_27_x86_64.whl

In [2]:
import pandas as pd
import xarray as xr
import pandera as pan
import optpricerpy
from optpricerpy.pricing_engine import (
    Measure,
    PricingEngine,
    ScenarioShift,
    RiskFactorFilter,
    MarketDataSchema
)
from datetime import date
import seaborn as sns

Let construct a small portfolio of option trades. 
 1. We first read a csv of trades into a `pandas` dataframe.
 2. Then we construct the `Portfolio` object. This python objects acts as a wrapper around a native portfolio struct defined in rust.
 3. We then load the option trades into the portfolio in one bulk operation.

In [3]:
option_trades_df = pd.read_csv("option_trades.csv")
portfolio = optpricerpy.portfolio.Portfolio()
portfolio.load_option_trades(option_trades_df)
option_trades_df

,ticker,size,strike,option_type,expiry
0,AAPL,3000,160,call,2022-02-01
1,AAPL,3500,130,put,2022-04-01
2,MSFT,500,400,call,2023-01-01
3,TSLA,400,1100,call,2022-01-22
4,TSLA,300,1300,call,2023-01-01
5,TSLA,600,800,put,2023-01-01


Note that we get input validation and the api checks that the structure of our dataframe conforms

In [4]:
try:
    portfolio.load_option_trades(option_trades_df.rename(columns={"strike": "stricke"}))
except pan.errors.SchemaError as e:
    print(e)

error in check_types decorator of function 'load_option_trades': column 'strike' not in dataframe
  ticker  size  stricke option_type      expiry
0   AAPL  3000      160        call  2022-02-01
1   AAPL  3500      130         put  2022-04-01
2   MSFT   500      400        call  2023-01-01
3   TSLA   400     1100        call  2022-01-22
4   TSLA   300     1300        call  2023-01-01


We initialize a `MarketData` object and bulk load data for the tickers

In [5]:
marketdata_df = MarketDataSchema.validate(
    pd.read_csv("market_data.csv").set_index(["ticker", "date"])
)
md = optpricerpy.pricing_engine.MarketData()
md.load_market_data(marketdata_df)
marketdata_df

spot       vol
ticker date                             
AAPL   2019-01-02    38.326290  0.470640
       2019-01-03    34.508717  0.430503
       2019-01-04    35.981865  0.729828
       2019-01-07    35.901775  0.704121
       2019-01-08    36.586166  0.640321
...                        ...       ...
TSLA   2021-11-22  1156.869995  0.634511
       2021-11-23  1109.030029  0.607280
       2021-11-24  1116.000000  0.622948
       2021-11-26  1081.920044  0.593477
       2021-11-29  1136.989990  0.593619

[2936 rows x 2 columns]

In [21]:
valdates = (
    marketdata_df.index.get_level_values("date")
    .unique()
    .sort_values()
    .to_pydatetime()[-10:]
)

The `PricingEngine` is the main entry to the calculation engine

In [22]:
pricing_engine = PricingEngine()

We can extract the full position image from the `Portfolio`.

In [23]:
sector_map = {
    "AAPL": "Technology",
    "MSFT": "Technology",
    "TSLA": "Automotive",
}
# augment the dataset with sector
posdata = portfolio.position_image().assign(sector=lambda x: x.ticker.map(sector_map))
posdata

,ticker,strike,expiry,option_type,sector
trade_id,,,,,
AAPL20220201C160_0,AAPL,160.0,2022-02-01,call,Technology
AAPL20220401P130_1,AAPL,130.0,2022-04-01,put,Technology
MSFT20230101C400_2,MSFT,400.0,2023-01-01,call,Technology
TSLA20220122C1100_3,TSLA,1100.0,2022-01-22,call,Automotive
TSLA20230101C1300_4,TSLA,1300.0,2023-01-01,call,Automotive
TSLA20230101P800_5,TSLA,800.0,2023-01-01,put,Automotive


Here we issue our first pricing request. We call into the engine requesting to price the supplied `Portfolio` object per the given daterange, using the supplied `MarketData`. 

In [24]:
metric_df = (
    pricing_engine.price_portfolio(
        [Measure.EXPOSURE, Measure.PRICE], valdates, portfolio, md
    )
    .unstack("measure")
    .droplevel(level=0, axis=1)
)
metric_df

measure                                 exposure       price
trade_id            valuation_date                          
AAPL20220201C160_0  2021-11-15      7.546674e+06    2.500465
                    2021-11-16      7.871344e+06    2.526947
                    2021-11-17      7.852819e+06    3.091205
                    2021-11-18      6.854182e+06    5.052587
                    2021-11-19      5.460479e+06    7.570962
                    2021-11-22      5.532273e+06    7.659242
                    2021-11-23      5.846278e+06    7.407307
                    2021-11-24      6.122004e+06    7.300942
                    2021-11-26      7.331433e+06    4.337704
                    2021-11-29      5.155874e+06    7.905157
AAPL20220401P130_1  2021-11-15     -7.363523e+06    1.382903
                    2021-11-16     -8.113598e+06    1.074148
                    2021-11-17     -9.206781e+06    0.710666
                    2021-11-18     -9.686078e+06    0.511288
                    2021-11-19     -7.769747e+06    0.845690
                    2021-11-22     -8.015052e+06    0.767749
                    2021-11-23     -8.939820e+06    0.560157
                    2021-11-24     -9.938547e+06    0.400594
                    2021-11-26     -9.677296e+06    0.536603
                    2021-11-29     -6.628808e+06    1.264666
MSFT20230101C400_2  2021-11-15      1.576834e+06    7.942358
                    2021-11-16      1.680690e+06    7.567781
                    2021-11-17      1.708372e+06    7.362657
                    2021-11-18      1.829832e+06    6.749876
                    2021-11-19      1.908397e+06    6.489634
                    2021-11-22      2.034779e+06    5.198163
                    2021-11-23      1.871852e+06    5.873667
                    2021-11-24      1.865878e+06    5.949317
                    2021-11-26      2.057016e+06    3.793247
                    2021-11-29      1.275985e+06   12.249711
TSLA20220122C1100_3 2021-11-15      1.945273e+06   98.255906
                    2021-11-16      2.034656e+06  110.983273
                    2021-11-17      2.072858e+06  125.026824
                    2021-11-18      2.148391e+06  123.665002
                    2021-11-19      2.230868e+06  139.473106
                    2021-11-22      2.282390e+06  147.060969
                    2021-11-23      2.451589e+06  112.793549
                    2021-11-24      2.404798e+06  118.602431
                    2021-11-26      2.609209e+06   93.053329
                    2021-11-29      2.568612e+06  121.217275
TSLA20230101C1300_4 2021-11-15      7.052628e+05  234.669264
                    2021-11-16      7.534279e+05  240.889021
                    2021-11-17      7.832977e+05  251.805334
                    2021-11-18      8.106396e+05  244.388478
                    2021-11-19      8.665469e+05  249.439184
                    2021-11-22      8.881883e+05  254.691323
                    2021-11-23      9.050889e+05  215.335355
                    2021-11-24      8.886810e+05  225.981386
                    2021-11-26      9.166897e+05  193.955604
                    2021-11-29      9.398491e+05  221.735591
TSLA20230101P800_5  2021-11-15     -7.954212e+05  188.352040
                    2021-11-16     -9.182211e+05  166.093499
                    2021-11-17     -1.006921e+06  152.532696
                    2021-11-18     -1.075893e+06  142.488568
                    2021-11-19     -1.248553e+06  121.665089
                    2021-11-22     -1.325011e+06  113.818213
                    2021-11-23     -1.315306e+06  113.947375
                    2021-11-24     -1.282871e+06  117.452483
                    2021-11-26     -1.308462e+06  114.297728
                    2021-11-29     -1.439304e+06  102.527690

Next we price the measure `exposure`, this time in a shifted context. We parameterize the calculation request with a volatility-shift

In [25]:
def _col_renamer(df):
    df.columns = [f"exposure_{c:.0%}" for c in df.columns]
    return df


vec_df = (
    pricing_engine.price_portfolio_ladder_scenario(
        [Measure.EXPOSURE],
        valdates,
        portfolio,
        md,
        ScenarioShift(
            Measure.VOL,
            None,
            rel_shifts=[0.0, 0.0, 0.0, 0.0],
            abs_shifts=[-0.05, -0.01, 0.01, 0.05],
        ),
    )
    .set_index(["trade_id", "valuation_date", "abs_shift"])
    .drop(columns=["measure", "rel_shift"])
    .unstack("abs_shift")
    .droplevel(level=0, axis=1)
    .pipe(_col_renamer)
)
vec_df.style.format(precision=0, thousands=" ")

Combine results to allow insights on a aggregated level

In [27]:
result_df = posdata.join(metric_df).join(vec_df)
result_df.style.format(precision=0, thousands=" ", formatter={"price": "{:0.2f}"})

In [28]:
exposure_cols = [
    "exposure_-5%",
    "exposure_-1%",
    "exposure",
    "exposure_1%",
    "exposure_5%",
]

Here we want to see the the portfolio sensitivity broken by underlying ticker

In [29]:
by_ticker = result_df.pivot_table(
    index=["ticker", "valuation_date"],
    values=["exposure", "exposure_-5%", "exposure_-1%", "exposure_1%", "exposure_5%"],
    aggfunc="sum",
)
by_ticker[exposure_cols].style.format(precision=0, thousands=" ")

or by sector

In [30]:
by_sector = result_df.pivot_table(
    index=["sector", "valuation_date"],
    values=["exposure", "exposure_-5%", "exposure_-1%", "exposure_1%", "exposure_5%"],
    aggfunc="sum",
)
by_sector[exposure_cols].style.format(precision=0, thousands=" ")

When we apply more advanced scenarios in more dimensions than 1 it is no longer practical to use a regular dataframe representation. The engine returns a more appropriate `xarray.Dataset` object (which can be thought of as a higher dimensional dataframe). 

In [66]:
%%timeit
arr = (
    pricing_engine.price_portfolio_2d_matrix_scenario(
        [Measure.EXPOSURE],
        valdates,
        portfolio,
        md,
        x_shift=ScenarioShift(
            Measure.PRICE,
            RiskFactorFilter.EQUITY,
            rel_shifts=[-0.05, -0.01, 0, 0.01, 0.05],
            abs_shifts=[0.0, 0.0, 0.0, 0.0, 0.0],
        ),
        y_shift=ScenarioShift(
            Measure.VOL,
            None,
            rel_shifts=[-0.05, -0.01, 0, 0.01, 0.05],
            abs_shifts=[0.0, 0.0, 0.0, 0.0, 0.0],
        ),
    )
    .merge(xr.Dataset(posdata))
    .assign_coords(
        {
            "shift_price": [-0.05, -0.01, 0, 0.01, 0.05],
            "shift_vol": [-0.05, -0.01, 0, 0.01, 0.05],
        }
    )
)
arr

1min 12s ± 3.33 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
arr.groupby("sector").sum()["exposure"].plot(col="sector")

ValueError: Only 1d and 2d plots are supported for facets in xarray. See the package `Seaborn` for more options.

### arr.groupby("ticker").sum()["exposure"].plot(col="ticker")

In [ ]:
arr["exposure"].sum("trade_id").plot()